In [201]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

class LobbyingDataPage:
    lobbying_file = 'lobbying/data/lobbying.csv'
    compensation_file = 'lobbying/data/compensation.csv'
    contributions_file = 'lobbying/data/contributions.csv'

    def __init__(self, html):
        self.html = html
        self.soup = bs(self.html,'html.parser')
        if self.isValid():

            self.is_entity = bool(self.soup.find('span', {'id': 'ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityCompany'}))

            self.company_name = self.get_company_name()
            self.date_range = self.get_date_range()

            if (self.soup.find('tr', {'class': 'GridHeader'})):
                self.lobbying_data = self.extract_lobbying_data()
                self.compensation_data = self.extract_compensation_data()
                self.contributions_data = self.extract_contributions_data()

        else:
            self.default_values()

    def isValid(self):
        if "An Error Occurred" in self.soup.text:
            return False

    def default_values(self):
            self.lobbying_data = pd.DataFrame()
            self.compensation_data = pd.DataFrame()
            self.contributions_data = pd.DataFrame()
            self.company_name = ''

    def get_date_range(self):
        return self.soup.find('span', {'id': 'ContentPlaceHolder1_lblYear'}).text

    def get_company_name(self):
        if self.is_entity:
            return self.soup.find('span', {'id': 'ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityCompany'}).text
        else:
            return self.soup.find('span', {'id': 'ContentPlaceHolder1_LRegistrationInfoReview1_lblLobbyistCompany'}).text

    def prep_tables(self):
        some_tables = self.soup.find_all('tr', {'style': 'vertical-align: top'})

        #Extract tables that contain the word 'lobbyist' and split at that word
        if 'Lobbyist name' in some_tables[0].text:
            split_tables = [table for table in some_tables if 'Client: ' in table.text][0].text.split('Client: ')
        else:
            split_tables = [table for table in some_tables if 'Lobbyist: ' in table.text][0].text.split('Lobbyist: ')
        #Strip out junk
        the_tables = [entry for entry in split_tables if entry.strip() and 'House / Senate' in entry]

        clean_tables = []
        for table in the_tables:
            clean_table = [line for line in table.split('\n') if line] # divide by lines and remove empties
            clean_table = clean_table[:clean_table.index('\xa0\xa0\xa0')] # Remove ending cruft
            clean_tables.append(clean_table)

        return clean_tables

    def extract_lobbying_data(self):
        if self.soup.find('span', {'id': 'ContentPlaceHolder1_LRegistrationInfoReview1_lblIncidental'}):
            return pd.DataFrame()
        clean_tables = self.prep_tables()
        row_dicts = []

        for table in clean_tables:
            lobbyist_name = table[0].strip()
            client_name = table[2].strip()
            table_start_index = table.index('House / SenateBill Number or Agency NameBill title or activityAgent positionAmountDirect business association')+1
            table_data = table[table_start_index:]

            i=0
            while i <= len(table_data)-8:
                row_dicts.append({'LobbyingEntity': self.company_name,
                                'DateRange': self.date_range,
                                'Lobbyist': lobbyist_name,
                                'Client': client_name,
                                'House/Senate': table_data[i].strip(),
                                'BillNumber':table_data[i+1].strip(),
                                'BillActivity':table_data[i+2].strip(),
                                'AgentPosition': table_data[i+3].strip(),
                                'Amount': table_data[i+5].strip(),
                                'DirectBusinessAssosciation': table_data[i+7].strip()})
                i=i+8
        return pd.DataFrame(row_dicts)

    def extract_contributions_data(self):

        bad_data = [element.split("Lobbyist: ")[0] for element in self.soup.text.split('Campaign Contributions') if "DateLobbyist nameRecipient nameOffice soughtAmount" in element]
        if not bad_data:
            print("NO DATA")
        pass1 = [element.split('Total contributions')[0] for element in bad_data]
        pass2 = [element.split('soughtAmount\n\n')[1:][0] for element in pass1]
        pass3 = "".join(pass2)
        data = [element.strip() for element in pass3.split('\n') if element.strip()]

        i = 0
        row_dicts = []
        while i < len(data):
            date = data[i].split()[0]
            lobbyist = " ".join(data[i].split()[1:])
            recipient = data[i+1]
            office = data[i+2]
            amount = data[i+3]
            row_dicts.append({  'LobbyingEntity': self.company_name,
                                'DateRange': self.date_range,
                                'Date': date,
                                'LobbyistName': lobbyist,
                                'RecipientName': recipient,
                                'OfficeSought': office,
                                'Amount': amount})
            i=i+4

        return pd.DataFrame(row_dicts)

    def extract_compensation_data(self):
        compensation_table = self.soup.find('table', {'id': 'ContentPlaceHolder1_DisclosureReviewDetail1_grdvClientPaidToEntity'})
        if not bool(compensation_table):
            return pd.DataFrame()
        temp_list = [line.strip() for line in compensation_table.text.split('\n') if line.strip()][1:-2]

        temp_dict_list = []
        for entry in temp_list:
            if entry[0] != '$':
                client_name = entry
            else:
                temp_dict_list.append({'LobbyingEntity': self.company_name, 'DateRange':self.date_range, 'Client': client_name, 'Amount':entry})
        return pd.DataFrame(temp_dict_list)

    def save(self):
        if not self.lobbying_data.empty:
            self.write_data(LobbyingDataPage.lobbying_file, self.lobbying_data)
        if not self.compensation_data.empty:
            self.write_data(LobbyingDataPage.compensation_file, self.compensation_data)
        if not self.contributions_data.empty:
            self.write_data(LobbyingDataPage.contributions_file, self.contributions_data)

    def write_data(self, file_path, dataframe):
        write = True
        #if os.path.exists(file_path):
        with open(file_path, mode = 'a', encoding = 'utf-8') as f:
            for line in f:
                if self.company_name in line and self.date_range in line:
                    print('Data already present in ' + file_path)
                    write = False
                    break

        if write and type(dataframe) == pd.DataFrame:
            print('Saving data to ' + file_path)
            dataframe.to_csv(file_path, mode ='a+',header=(not os.path.exists(file_path)), index=False)

In [4]:
def extract_and_save(html_list):
    #for html in html_list:
        #LobbyingDataPage(html).save()
    for i in range(len(html_list)):
        print("Saving "+str(i))
        LobbyingDataPage(html_list[i]).save()

def pull_data(url):
    headers={"User-Agent": "Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148"}
    result = requests.get(url, headers=headers)
    result.raise_for_status()
    return result.content

def download_html_list(url_list):
    html_list = []
    for url in url_list:
        print("Pulling data from " + url)
        html_list.append(pull_data(url))
    return html_list

def save_data_from_url_list(url_list):
    disclosure_links = extract_and_save(download_html_list(url_list))
    html_list = download_html_list(disclosure_links)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def extract_client_links(year):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://www.sec.state.ma.us/LobbyistPublicSearch/Default.aspx'

    driver.get(url)

    driver.find_element('id','ContentPlaceHolder1_rdbSearchByType').click()
    select = Select(driver.find_element(By.CLASS_NAME,'p3'))

    select.select_by_value(year)
    Select(driver.find_element('id','ContentPlaceHolder1_ucSearchCriteriaByType_drpType')).select_by_value('L')
    driver.find_element('id','ContentPlaceHolder1_btnSearch').click()

    find_table = driver.find_element(By.ID,'ContentPlaceHolder1_ucSearchResultByTypeAndCategory_grdvSearchResultByTypeAndCategory')
    links = find_table.find_elements(By.TAG_NAME,'a')
    links_list = [l.get_attribute('href') for l in links if str(l.get_attribute('href')).startswith('javascript') == False]
    driver.quit()
    return links_list

def extract_disclosures(list_of_links):
    disclosure_reports = []

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for link in list_of_links:
    # print(link)
        driver.get(link)
        all_links = driver.find_elements(By.CLASS_NAME,'BlueLinks')
        disclosure_links = [l.get_attribute('href') for l in all_links if 'CompleteDisclosure' in l.get_attribute('href')]
        for dl in disclosure_links:
            disclosure_reports.append(dl)
    driver.quit()

    return disclosure_reports


In [119]:
import pickle
with open("2005html.pkl", "rb") as f:
    html05 = pickle.load(f)
with open("2020html.pkl", "rb") as f:
    html20 = pickle.load(f)

In [39]:
import pickle
with open("indivlobbyist.html", "w") as f:
    f.write(html05[0].decode('utf-8'))

In [1]:
test_entity_urls = ['https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oUX2F0/qMX8aZhXGSqISnPo81sWNBWPRVYkBCJOoiSOC',
    'https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oSXfp14ycsC4C75XzUXuOD0RNTxP5RQlQYtqqNlG19gK',
    'https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=Kce7BzXCV/xrL2hRhIeiyrKq4598/MmeOqNxcRw3anF8llP1KzXu6cA+wFHr/nIU',
    'https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=Tcg7Il3rjW5sIbUrwbcVKYqHMk7FN1E+JyuG2w4SuGbSUM5P5U7i1R+Kl69eLgqM']

test_lobbyist_urls = ['https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oazP9bD0a9KMVAPrqT2Yinwr4JTgsyzaInIK/BXJHlV1',
    'https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oWE66BrPrRKWkGd1M0SOekxiCPdVzrEEIQIimWwrunVO',
    'https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=v/mjLQ41YVqm2bof1TANC8QjRgi8rX5lY/Ozmu5hJvE2+nv22rfxUQCNlsde/z4F',
    'https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=qOH5OAu6URrG3qvY0KcrjT8Cd6HIk4OEVgmMDn8i9vU6n8cVsZ6PiBz3uD4tmhUG']

In [6]:
test_entity_htmls = download_html_list(test_entity_urls)
test_lobbyist_htmls = download_html_list(test_lobbyist_urls)

Pulling data from https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oUX2F0/qMX8aZhXGSqISnPo81sWNBWPRVYkBCJOoiSOC
Pulling data from https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oSXfp14ycsC4C75XzUXuOD0RNTxP5RQlQYtqqNlG19gK
Pulling data from https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=Kce7BzXCV/xrL2hRhIeiyrKq4598/MmeOqNxcRw3anF8llP1KzXu6cA+wFHr/nIU
Pulling data from https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=Tcg7Il3rjW5sIbUrwbcVKYqHMk7FN1E+JyuG2w4SuGbSUM5P5U7i1R+Kl69eLgqM
Pulling data from https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oazP9bD0a9KMVAPrqT2Yinwr4JTgsyzaInIK/BXJHlV1
Pulling data from https://www.sec.state.ma.us/LobbyistPublicSearch/CompleteDisclosure.aspx?sysvalue=eaiHDZ6kDM3fHlDyBbc8oWE66BrPrRKWkGd1M0SOekxiCPdVzrEEIQIimWwrunVO
Pulling da

NameError: name 'html_list' is not defined

In [274]:
edf = pd.read_html(test_entity_htmls[0])
ldf = pd.read_html(test_lobbyist_htmls[0])

In [351]:
[df for df in edf[5:] if 'Client Compensation' in str(df) and len(df) == 2][0][0][1]


'NameAmount  Massachusetts Business Alliance for Education  $6,000.00  The Education Trust Inc  $3,900.00  Total salaries received  $9,900.00'

In [356]:
edf[7][0][1]

'NameAmount  Massachusetts Business Alliance for Education  $6,000.00  The Education Trust Inc  $3,900.00  Total salaries received  $9,900.00'

In [303]:
edf[10]

,0,1,2,3,4,5
0,"No activities, bill numbers or bill titles wer...","No activities, bill numbers or bill titles wer...","No activities, bill numbers or bill titles wer...","No activities, bill numbers or bill titles wer...","No activities, bill numbers or bill titles wer...","No activities, bill numbers or bill titles wer..."


In [117]:
ldf[8][0][0].split('Client: ')[1]

'Massachusetts Medical Society'

In [265]:
table = ldf[5][0:7].transpose()
table.columns = table.iloc[0]
table = table[1:]
name = table['Lobbyist name'][1]
re.sub("\s\s+", " ", name)
name

'Leda  Anderson'

In [391]:
class LobbyingDataPage:
    def __init__(self, html):
        self.tables = {}
        self.dfs = pd.read_html(html)
        if 'An Error Occurred' in str(self.dfs[0][0]):
            #end this. Set default values?
            pass
        else:
            self.is_entity = 'Entity' in self.dfs[4][0][2]
            self.get_date_range()
            self.get_header()
            self.scrape_tables()

    def get_date_range(self):
        self.date_range = self.dfs[4][0][2].split('period:  ')[1]

    #Extracts the table of header info from the top of the page
    #TODO merge entites and lobbyists tables??
    # add Lobbyist/Entity column
    # fill out n/a fields with n/a's
    def get_header(self):
        table = self.dfs[5][0:7].transpose()
        table.columns = table.iloc[0]
        table = table[1:]
        if self.is_entity:
            self.tables['Entities'] = table
        else:
            self.tables['Lobbyists'] = table

    def scrape_tables(self):
        for i in range(len(self.dfs[5:])):
            df_str = str(self.dfs[i])
            #ACTIVITIES TABLES
            if 'House / Senate' in df_str and len(self.dfs[i]) == 1:
                self.get_activities(i)
            
            #CLIENT COMPENSATION
            if 'Client Compensation' in df_str and len(self.dfs[i]) == 2:
                self.get_compensation(i)

    def get_activities(self, i):
        self.tables.setdefault('Activities', pd.DataFrame()) #Create table if it doesn't exist
        client = str(self.dfs[i-1][0][0]).split('Client:')[1].strip()
        if self.is_entity:
            lobbyist = self.dfs[i-2][0][0].split('Lobbyist:')[1].strip()
        else:
            lobbyist = str(self.tables['Lobbyists']['Lobbyist name'][1])
        table = self.dfs[i+1][:-1]
        table.insert(0, 'Client', client)
        table.insert(0, 'Lobbyist', lobbyist)
        table.insert(0, 'Date Range', self.date_range)
        self.tables['Activities'] = pd.concat( [self.tables['Activities'], table])

    def get_compensation(self, i):
        self.tables.setdefault('Compensation', pd.DataFrame())
        comp_str = self.dfs[i][0][1]
        data = re.findall(r'[\w\s\.&,]+\s\$[\d,\.]+', comp_str[11:])
        data = [d.split(" $") for d in data]
        data = [[d[0], float(d[1].replace(',',''))] for d in data if len(d) == 2]
        table = pd.DataFrame(data, columns = ['Name', 'Amount'])
        self.tables['Compensation'] = pd.concat( [self.tables['Compensation'], table])

    def get_salaries(self, i):
        pass

    def get_operating_expenses(self, i):
        pass

    def get_entertainment_expenses(self, i):
        pass

    def get_campaign_contributions(self, i):
        pass

    def clean_entry(entry):
        return re.sub("\s\s+", " ", entry)

    def fetch_tables(self):
        return self.tables
        
    def save(self):
        for table in self.tables.keys():
            self.write_data(f'{table.replace(" ","_").lower()}.csv', self.tables[table])

    def write_data(self, file_path, dataframe):
        write = True
        #if os.path.exists(file_path):
        with open(file_path, mode = 'a', encoding = 'utf-8') as f:
            for line in f:
                if self.company_name in line and self.date_range in line:
                    print('Data already present in ' + file_path)
                    write = False
                    break

        if write and type(dataframe) == pd.DataFrame:
            print('Saving data to ' + file_path)
            dataframe.to_csv(file_path, mode ='a+',header=(not os.path.exists(file_path)), index=False)

In [386]:
dfs = pd.read_html(test_entity_htmls[1])
comp_str = dfs[7][0][1]
#df = pd.DataFrame([dfs[7][0][1].split("$")[1:-2:2], dfs[7][0][1].split("$")[2:-2:2]]).transpose()
#df.columns = ['Name','Amount']
comp_str

'NameAmount  PhysicianOne Urgent Care  $61,674.00  ZAFA XXV, LLC  $30,000.00  Massachusetts Aggregate and Asphalt Pavements Association a Division of Construction Industries of Massachusetts  $21,000.00  USA Hauling & Recycling, Inc.  $0.00  Blue Tarp reDevelopment, LLC dba MGM Springfield  $25,000.00  Manafort Brothers Incorporated  $22,916.63  American Medical Response  $60,506.25  Massachusetts Association of Insurance Agents, Inc.  $67,500.00  Massachusetts Municipal Wholesale Electric Company  $54,999.96  Whalley Computer Associates, Inc. $25,000.00  Massachusetts Alliance of Boys & Girls Clubs  $60,166.50  Irish Cultural Center Inc. of Western New England  $4,166.66  Total salaries received  $432,930.00'

In [390]:
def string_to_dataframe(string):
    data = re.findall(r'[\w\s\.&,]+\s\$[\d,\.]+', string[11:])
    data = [d.split(" $") for d in data]
    data = [[d[0], float(d[1].replace(',',''))] for d in data if len(d) == 2]
    df = pd.DataFrame(data, columns = ['Name', 'Amount'])
    return df

string_to_dataframe(comp_str)

,Name,Amount
0,PhysicianOne Urgent Care,61674.00
1,"ZAFA XXV, LLC",30000.00
2,Massachusetts Aggregate and Asphalt Pavement...,21000.00
3,"USA Hauling & Recycling, Inc.",0.00
4,"Blue Tarp reDevelopment, LLC dba MGM Springf...",25000.00
5,Manafort Brothers Incorporated,22916.63
6,American Medical Response,60506.25
7,Massachusetts Association of Insurance Agent...,67500.00
8,Massachusetts Municipal Wholesale Electric C...,54999.96
9,"Whalley Computer Associates, Inc.",25000.00


In [400]:
LobbyingDataPage(test_lobbyist_htmls[2]).fetch_tables()['Compensation']

KeyError: 'Compensation'

In [181]:
tables = {}
for i in range(len(edf)):
    if ('House / Senate' in str(edf[i]) and len(edf[i]) == 1):
        print(i)
        tables.setdefault('Activities', pd.DataFrame())
        client = edf[i-1][0][0].split('Client:')[1].strip()
        table = edf[i+1][:-1]
        tables['Activities'] = pd.concat( [tables['Activities'], edf[i+1][:-1]])

17
21


In [168]:
for i in range(len(html05)):
    dfs = pd.read_html(html05[i])
    is_error = 'An Error Occurred' in str(dfs[0][0])
    if is_error:
        print(f'Error processing index {i}')
        #Return
    else: #Remove
        is_entity = 'Entity' in dfs[4][0][2]
        header_table = dfs[5]

Error processing index 48
Error processing index 54


In [ ]:
# Header:
# Authorizing Officer name / Lobbyist name
# Title / NA
# Business name / Business name
# Address / Address
# City, state, zip code / Citym, state, zip code
# country / country
# NA / Agent Type

#TABLES:
# Entities
# Lobbyists
# Client Compensation
# Compensation/Salaries Paid
# Activities, Bill Numbers and Titles
# Operating Expenses
# Meals, Travel, and Entertainment Expenses
# Additional Expenses
# Campaign Contributions